In [1]:
using NeuralPDE, Lux, Plots, Random, Distributions, Optim , OptimizationOptimJL
using IntegralsCubature, FinancialToolbox
using ModelingToolkit: Interval
Random.seed!(0)



TaskLocalRNG()

In [2]:
σ_min = 1.5
σ_max = 2.5
r_min = 0.05
r_max = 0.15
σ_mid = (σ_min + σ_max)/2
r_mid = (r_min + r_max)/2
α = 0.5
K = 100 # strike is 100
T = 1 #one year 
Smin = 80
Smax = 120 # stock range make it 20% in-the-money and out-the-money
scaling_factor = 0.005
boundary_func(s) = max(s - K,0)*scaling_factor

boundary_func (generic function with 1 method)

In [3]:
κ(σ,r,tau) = 2*r/(σ^(2*(1 - α))*(exp(2*r*(1-α)*tau)-1))
a(x,σ,r,tau) = κ(σ,r,tau)*x.^(2*(1 - α))*exp(2*r*(1 - α)*tau)
b(σ,r,tau) = κ(σ,r,tau)*K^(2*(1 - α))
c = 2 + (1/(1 - α))
F(x,σ,r,tau) = Distributions.NoncentralChisq(c,a(x,σ,r,tau))
G(σ,r,tau) = Distributions.NoncentralChisq(c - 2,b(σ,r,tau))
function CEV_call(x,σ,r,tau)
    x*(1 - Distributions.cdf(F(x,σ,r,tau), b(σ,r,tau))) - K*exp(-r*tau)*Distributions.cdf(G(σ,r,tau), a(x,σ,r,tau))
end

CEV_call (generic function with 1 method)

In [4]:
# 4D PDE in (t, s)
@parameters t s σ r
@variables U(..)
Dt = Differential(t)
Ds = Differential(s)
Dss = Differential(s)^2

eq = Dt(U(t, s, σ, r)) + r*s*Ds(U(t, s, σ, r)) + 1/2*σ^2*s^(2*α)*Dss(U(t, s, σ, r)) - r*U(t, s, σ, r) ~ 0

# Boundary Conditions
boundary_conditions = [U(T, s, σ, r) ~ boundary_func(s)]

# Problem Domain
domains = [t ∈ Interval(0, T), s ∈ Interval(Smin, Smax),
           σ ∈ Interval(σ_min,σ_max), r ∈ Interval(r_min,r_max)]

4-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(t, 0..1)
 Symbolics.VarDomainPairing(s, 80..120)
 Symbolics.VarDomainPairing(σ, 1.5..2.5)
 Symbolics.VarDomainPairing(r, 0.05..0.15)

In [5]:
dim = 4
chain = Lux.Chain(Dense(dim, 16, Lux.σ), Dense(16, 16, Lux.σ), Dense(16, 16, Lux.σ), Dense(16, 1))
ps = Lux.setup(Random.default_rng(), chain)[1]

# Transform PDESystem into OptimizationProblem using PINN methodology
strategy = QuadratureTraining()
#strategy = QuadratureTraining(quadrature_alg = CubatureJLh(),
                     #reltol = 1e-6, abstol = 1e-3,
                     #maxiters = 2_500, batch = 500)
discretization_algo = PhysicsInformedNN(chain, strategy, init_params=ps)
@named pde_system = PDESystem(eq, boundary_conditions, domains, [t, s, σ, r], [U(t, s, σ, r)])
optim_prob = discretize(pde_system, discretization_algo)

OptimizationProblem. In-place: true
u0: ComponentVector{Float32}(layer_1 = (weight = Float32[-0.48351604 0.40789896 0.40354213 0.099549524; -0.049033877 -0.18695378 -0.421895 -0.093530044; … ; -0.22635894 4.831728f-6 0.23992884 -0.3143545; 0.52216566 -0.032581452 -0.35231933 0.07687697], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = Float32[0.15251705 0.39284882 … -0.022368534 0.20915501; -0.2943246 0.08905726 … 0.2225429 -0.1315318; … ; -0.42614084 -0.41936278 … -0.3245835 -0.14442606; 0.047960266 0.27465823 … 0.005657302 0.058415208], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = Float32[0.28811473 0.36900693 … -0.37211332 0.019529067; 0.14447463 -0.24254143 … -0.29063964 0.09696433; … ; -0.39559042 0.25605455 … -0.063562825 -0.16985989; 0.1261293 -0.20957911 … 0.052887935 0.37911317], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_4 = (weight = Float32[-0.45333603 -0.07046209 … 0.20396504 -0.3476128], bias = Float32[0.0;;]))

In [6]:
using TickTock

In [ ]:
tick()

callback = function (p,l)
    println("Current loss is: $l")
    return false
end


opt = BFGS()
result = Optimization.solve(optim_prob, opt, callback = callback, maxiters=2500, reltol=1e-6)
result.original
phi = discretization_algo.phi

tock()

[ Info:  started timer at: 2023-01-26T11:49:46.814


Current loss is: 0.04761720036606123
Current loss is: 0.04672005433908653
Current loss is: 0.046523851962177586
Current loss is: 0.046436025821978255
Current loss is: 0.046272887140173566
Current loss is: 0.045908099585647
Current loss is: 0.04583506757491798
Current loss is: 0.045662065885278144
Current loss is: 0.04516464901436386
Current loss is: 0.04485483584303401
Current loss is: 0.04482317885565285
Current loss is: 0.044744808035205905
Current loss is: 0.044437752631579945
Current loss is: 0.04433120267199507
Current loss is: 0.04363869920482783
Current loss is: 0.042394811519040496
Current loss is: 0.03697802133514423
Current loss is: 0.03664337765655872
Current loss is: 0.034531967825945455
Current loss is: 0.030169939603856864
Current loss is: 0.02870300311849176
Current loss is: 0.02865626323571464
Current loss is: 0.02588260019287434
Current loss is: 0.024532287386413793
Current loss is: 0.024192071581487826
Current loss is: 0.022852969270180604
Current loss is: 0.022479956

In [ ]:
dx = 0.5
x_domain = Smin:dx:Smax
S_domain = collect(x_domain)

times = [0, 0.25, 0.5, 1]
tau = T .- times
plots = Array{Plots.Plot{Plots.GRBackend},1}()
for i in 1:(length(times))-1
    current_t = times[i]
    current_tau = tau[i]
    u_predict = [phi([current_t, current_x, σ_mid, r_mid],result.u) for current_x in x_domain]
    u_predict = reduce(vcat, u_predict)/scaling_factor
    u_true = CEV_call.(S_domain, σ_mid, r_mid, current_tau)
    push!(plots, plot(S_domain, [u_true, u_predict], label= ["CEV_True" "PINN"],legend=:topleft, title="Tau = $(current_tau)"))
    #push!(plots, plot(S_domain, V_predict, label=""))
end

u_predict = [phi([times[4], current_x, σ_mid, r_mid],result.u) for current_x in x_domain]
u_predict = reduce(vcat, u_predict)/scaling_factor


u_true = collect(max.(S_domain .- K,0))
u_true = reduce(vcat, u_true)

push!(plots, plot(S_domain, [u_true, u_predict], label= ["CEV_True" "PINN"],legend=:topleft, title="Tau = 0"))



plot(plots[1], plots[2], plots[3], plots[4], layout = 4)

In [ ]:
dx = 0.5
x_domain = Smin:dx:Smax
S_domain = collect(x_domain)
#S_domain = Float32.(S_domain)


times = [0, 0.25, 0.5, 1]
#plots = Array{Plots.Plot{Plots.GRBackend},1}()

u_predict1 = [phi([times[1], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict1 = reduce(vcat, u_predict1)/scaling_factor
u_true1 = CEV_call.(S_domain, σ_mid, r_mid, tau[1])
u_predict2 = [phi([times[2], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict2 = reduce(vcat, u_predict2)/scaling_factor
u_true2 = CEV_call.(S_domain, σ_mid, r_mid, tau[2])
u_predict3 = [phi([times[3], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict3 = reduce(vcat, u_predict3)/scaling_factor
u_true3 = CEV_call.(S_domain, σ_mid, r_mid, tau[3])
u_predict4 = [phi([times[4], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict4 = reduce(vcat, u_predict4)/scaling_factor
u_true4 = collect(max.(S_domain .- K,0))
u_true4 = reduce(vcat, u_true4)


In [ ]:
using GLMakie
using ColorSchemes
GLMakie.activate!()

In [ ]:
using PyPlot

fig = figure(figsize=(10, 10), dpi=1200);
#subplots_adjust(hspace=0.5,wspace=0.9)
subplot(221) ;  PyPlot.plot(x_domain,u_predict1,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true1,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(222) ;  PyPlot.plot(x_domain,u_predict2,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true2,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.25",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(223) ;  PyPlot.plot(x_domain,u_predict3,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true3,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.5",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(224) ;  PyPlot.plot(x_domain,u_predict4,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true4,color = "magenta", label = "Analytical");
PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

fig
save("./Fig_7.png", fig)

In [ ]:


ts = collect(range(0,T,length=100))
xs = collect(range(Smin,Smax,length=100))
σs = collect(range(σ_min,σ_max,length=100))
rs = collect(range(r_min,r_max,length=100))

meshgrid(x, ys) = [x[i] for i in 1:length(x), j in length(ys)], [ys[j] for j in 1:length(ys), i in 1:length(x)]
domain_x, domain_t = meshgrid(xs,ts)    #use domain_t as meshgrid for time
domain_y2, domain_s = meshgrid(ts,xs)   #use domain_s as meshgrid for stock prices
domain_x, domain_σ = meshgrid(xs,σs)    #use domain_σ as meshgrid for σ range
domain_x, domain_r = meshgrid(xs,rs)    #use domain_r as meshgrid for r range 

In [ ]:
predict_matrix1 = zeros(length(ts),length(xs))
analytical_matrix1 = zeros(length(ts),length(xs))
for i in 1:length(ts)
    time = ts[i]
    tau_val = T - time
    predict1 = [phi([time, stock, σ_mid, r_mid],result.u) for stock in xs]
    predict1 = reduce(vcat, predict1)/scaling_factor
    u_true1 = [CEV_call.(ss, σ_mid, r_mid, tau_val) for ss in xs]
    predict_matrix1[:,i] = predict1
    analytical_matrix1[:,i] = u_true1
end

In [ ]:
predict_matrix2 = zeros(length(σs),length(xs))
analytical_matrix2 = zeros(length(σs),length(xs))
for i in 1:length(σs)
    current_σ = σs[i]
    predict2 = [phi([0, stock, current_σ, r_mid],result.u) for stock in xs]
    predict2 = reduce(vcat, predict2)/scaling_factor
    u_true2 = [CEV_call.(ss, current_σ, r_mid, tau[1]) for ss in xs]
    predict_matrix2[:,i] = predict2
    analytical_matrix2[:,i] = u_true2
end

In [ ]:
# first plot

 
#using LaTeXStrings

fig = GLMakie.Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = "Call Price",
    title = "CEV PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = " Call Price",
    title = " CEV Analytical solution")

GLMakie.surface!(ax1, domain_s , domain_t', predict_matrix1 , transparency = true)
GLMakie.surface!(ax2, domain_s, domain_t', analytical_matrix1 , transparency = true)
save("./Fig_8.png", fig)
fig

In [ ]:
#second plot


fig = GLMakie.Figure(resolution=(1200, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = "Absolute error",
    title = "Absolute error")


GLMakie.surface!(ax1, domain_s , domain_t', abs.(analytical_matrix1 .- predict_matrix1) , transparency = true)

save("./Fig_9.png", fig)
fig

In [ ]:

# Third plot

#solar = ColorSchemes.solar.colors
fig = GLMakie.Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = "Call Price",
    title = "CEV PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = " Call Price",
    title = " CEV Analytical")

GLMakie.surface!(ax1, domain_s , domain_σ', predict_matrix2 , transparency = true, colormap = ColorSchemes.autumn1)
GLMakie.surface!(ax2, domain_s, domain_σ', analytical_matrix2 , transparency = true, colormap = ColorSchemes.autumn1)
save("./Fig_12.png", fig)
fig

In [ ]:
# Fourth plot

fig = GLMakie.Figure(resolution=(1200, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = "Absolute error",
    title = "Absolute error")


GLMakie.surface!(ax1, domain_s , domain_σ', abs.(analytical_matrix2 .- predict_matrix2) , transparency = true,
                    colormap = ColorSchemes.autumn1)

save("./Fig_13.png", fig)
fig